In [1]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/Users/wilsonchang/Desktop/Project/2ndtest/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helper Clases / Functions

In [2]:
def checkAccuracy(model, dataloader):
    y_pred_tot = []
    y_sample_tot = []
    for i_batch, sample_batched in enumerate(dataloader):
        X_sample, y_sample = sample_batched
        # Flip axis first
        X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
        X_sample, y_sample = Variable(X_sample), Variable(y_sample)
        scores = model(X_sample)
        #loss = loss_fn(scores, y_sample.long())
        #print('val loss: %f' % loss)
        _, y_pred = torch.max(scores, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_sample_tot.append(y_sample.data.numpy())
    acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_sample_tot))
    return acc
class Flatten(nn.Module):
    def forward(self, x):
        out, _ = x
        T, N, H = out.size() 
        return out.view(H, -1)

# Load Data

In [3]:
A01T = h5py.File('./../project_datasets/A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
X = X[:, 0:23, :] # only goes to 23 since you dont use last 3 channels
# X = np.moveaxis(X, -1, 0)
y = y[0,0:X.shape[0]:1]
#subtract 769 so classes start at 0
y = y - 769
N, E, T = np.shape(X)
print (np.shape(X))

<HDF5 file "A01T_slice.mat" (mode r)>
(288, 23, 1000)


# Make DataLoaders

In [6]:
bs = 50
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=bs, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Define Model

In [7]:
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layer)
                            # batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        # h0 = Variable(torch.zeros(self.n_layer, x.size(1),
                                #   self.hidden_dim)).cuda()
        # c0 = Variable(torch.zeros(self.n_layer, x.size(1),
                                #   self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[-1, :, :]
        out = self.classifier(out)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample)
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.numpy() == y_sample.data.numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

# Test the Model

In [8]:
dtype = torch.FloatTensor
hidden_dim = 100
num_classes = 4
num_epoches = 10
model = myLSTM(E, hidden_dim, 2, num_classes)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train)
        # print (out)
        loss = loss_fn(out, y_train.long())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
        

(0 batch) loss: 1.391356
(1 batch) loss: 1.382428
(2 batch) loss: 1.392851
(3 batch) loss: 1.379888
(Epoch 1 / 10) train_acc: 0.388298; val_acc: 0.260000
(0 batch) loss: 1.354640
(1 batch) loss: 1.362906
(2 batch) loss: 1.351139
(3 batch) loss: 1.348818
(Epoch 2 / 10) train_acc: 0.377660; val_acc: 0.300000
(0 batch) loss: 1.298985
(1 batch) loss: 1.366675
(2 batch) loss: 1.345711
(3 batch) loss: 1.331232
(Epoch 3 / 10) train_acc: 0.393617; val_acc: 0.280000
(0 batch) loss: 1.340405
(1 batch) loss: 1.322417
(2 batch) loss: 1.334544
(3 batch) loss: 1.240284
(Epoch 4 / 10) train_acc: 0.436170; val_acc: 0.300000
(0 batch) loss: 1.303195
(1 batch) loss: 1.301097
(2 batch) loss: 1.240848
(3 batch) loss: 1.323014
(Epoch 5 / 10) train_acc: 0.500000; val_acc: 0.220000
(0 batch) loss: 1.265337
(1 batch) loss: 1.228786
(2 batch) loss: 1.299441
(3 batch) loss: 1.235741
(Epoch 6 / 10) train_acc: 0.553191; val_acc: 0.140000
(0 batch) loss: 1.191869
(1 batch) loss: 1.255946
(2 batch) loss: 1.222051
(